<a href="https://colab.research.google.com/github/ghommidhWassim/GNN-variants/blob/main/new_test_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git
!python -c "import torch; print(torch.__version__)"
!python -c "import torch; print(torch.version.cuda)"
!pip install torchvision
!pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.6.0+cu124.html


  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
2.6.0+cu124
12.4
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 62.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 837.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 81.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
   

In [23]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import scipy.sparse as sp
from torch_geometric.datasets import Planetoid, Amazon
from torch_geometric.transforms import NormalizeFeatures, RandomNodeSplit
import torch_geometric.transforms as T
from torch_geometric.utils import to_scipy_sparse_matrix, to_networkx
from sklearn.metrics import accuracy_score
import random
from sklearn.metrics import f1_score
import json,time
import gc
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import networkx as nx # Import networkx

In [3]:
class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super().__init__()
        self.convs = nn.ModuleList()
        self.convs.append(nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(nn.Linear(hidden_channels, hidden_channels))
        self.convs.append(nn.Linear(hidden_channels, out_channels))

    def forward(self, x, adjs):
        for i, (conv, adj) in enumerate(zip(self.convs[:-1], adjs)):
            x = conv(x)
            x = torch.sparse.mm(adj, x)
            x = F.relu(x)
        x = self.convs[-1](x)
        return F.log_softmax(x, dim=-1)

In [4]:
import numpy as np
import networkx as nx
from sklearn.metrics.pairwise import cosine_similarity

def build_subgraphs_by_feature_correlation(adj_matrix, features, num_neighbors=4, num_subgraphs=128):
    num_nodes = features.shape[0]
    G = nx.from_numpy_array(adj_matrix)
    visited = set()
    subgraphs = []
    subgraph_node_lists = []

    feature_sim = cosine_similarity(features)  # shape: [N, N]

    node_indices = np.random.permutation(num_nodes)

    for node in node_indices:
        if node in visited:
            continue

        neighbors = list(G.neighbors(node))
        if len(neighbors) == 0:
            continue

        # Keep neighbors that are unvisited
        neighbors = [n for n in neighbors if n not in visited]
        if not neighbors:
            continue

        # Rank neighbors by *lowest* feature correlation
        correlations = feature_sim[node, neighbors]
        sorted_neighbors = [n for _, n in sorted(zip(correlations, neighbors))][:num_neighbors]

        sub_nodes = [node] + sorted_neighbors
        sub_nodes = list(set(sub_nodes))  # remove any possible duplicates

        # Mark as visited
        visited.update(sub_nodes)
        subgraph_node_lists.append(sub_nodes)

        # Create subgraph adjacency
        sub_adj = adj_matrix[np.ix_(sub_nodes, sub_nodes)]
        subgraphs.append(sub_adj)

        if len(subgraphs) >= num_subgraphs:
            break

    return subgraphs, subgraph_node_lists


In [6]:
def build_inter_subgraph_matrix(subgraph_node_lists, adj_matrix):
    num_subgraphs = len(subgraph_node_lists)
    inter_matrix = np.zeros((num_subgraphs, num_subgraphs))

    for i in range(num_subgraphs):
        for j in range(i+1, num_subgraphs):
            nodes_i = subgraph_node_lists[i]
            nodes_j = subgraph_node_lists[j]
            # Count interconnections
            inter_links = adj_matrix[np.ix_(nodes_i, nodes_j)]
            if np.any(inter_links):
                inter_matrix[i, j] = inter_matrix[j, i] = 1

    return inter_matrix


class CustomSubgraphDataset(torch.utils.data.Dataset):
    def __init__(self, subgraphs, node_lists, features, labels):
        self.subgraphs = subgraphs
        self.node_lists = node_lists
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.subgraphs)

    def __getitem__(self, idx):
        nodes = self.node_lists[idx]
        sub_x = self.features[nodes]
        sub_y = self.labels[nodes]
        sub_adj = self.subgraphs[idx]

        sub_adj = torch.FloatTensor(sub_adj)
        sub_x = torch.FloatTensor(sub_x)
        sub_y = torch.LongTensor(sub_y)

        return sub_x, sub_adj, sub_y


In [26]:
def train(model, data, optimizer, criterion, subgraphs, device):
    model.train()
    total_loss = 0

    for sub_nodes in subgraphs:
        # Mapping from global node index → local index in subgraph
        node_map = {int(n): i for i, n in enumerate(sub_nodes)}
        sub_nodes = torch.tensor(sub_nodes, dtype=torch.long)

        # Build local edge_index: filter only edges inside the subgraph
        sub_adj = data.edge_index.cpu().numpy()  # shape [2, num_edges]
        sub_nodes_set = set(sub_nodes.tolist())
        filtered_edges = []

        for src, dst in sub_adj.T:
            if src in sub_nodes_set and dst in sub_nodes_set:
                filtered_edges.append([node_map[src], node_map[dst]])

        if len(filtered_edges) == 0:
            continue  # skip subgraph with no internal edges

        # Build edge_index tensor [2, num_edges]
        sub_edge_index = torch.tensor(filtered_edges, dtype=torch.long).T.to(device)

        # Subgraph features and labels
        x_sub = data.x[sub_nodes].to(device)
        y_sub = data.y[sub_nodes].to(device)

        # Forward pass
        optimizer.zero_grad()
        out = model(x_sub, sub_edge_index)

        # Loss: only on training nodes inside the subgraph
        train_mask_sub = data.train_mask[sub_nodes].to(device)
        if train_mask_sub.sum() == 0:
            continue  # skip if no training nodes in this subgraph

        loss = criterion(out[train_mask_sub], y_sub[train_mask_sub])
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    return total_loss / len(subgraphs)



from sklearn.metrics import accuracy_score, f1_score

def test(model, data, device):
    model.eval()
    with torch.no_grad():
        x = data.x.to(device)
        edge_index = data.edge_index.to(device)
        values = torch.ones(edge_index.shape[1]).to(device)
        adj = torch.sparse_coo_tensor(edge_index, values, size=(x.size(0), x.size(0))).to(device)

        out = model(x, [adj] * (len(model.convs) - 1))
        pred = out.argmax(dim=1)
        true = data.y.to(device)

        acc = accuracy_score(true.cpu(), pred.cpu())
        f1 = f1_score(true.cpu(), pred.cpu(), average='micro')

        return acc, f1


In [18]:
def build_subgraphs(data, subgraph_size=5, num_neighbors=4):
    G = to_networkx(data, to_undirected=True)
    features = data.x.cpu().numpy()
    adj = nx.to_numpy_array(G)
    visited = set()
    subgraphs = []
    subgraph_connections = []

    # Node-to-node correlation matrix
    similarity = cosine_similarity(features)

    while len(visited) < data.num_nodes:
        available_nodes = list(set(range(data.num_nodes)) - visited)
        if not available_nodes:
            break

        start = np.random.choice(available_nodes)
        current = [start]
        visited.add(start)

        neighbors = list(G.neighbors(start))
        neighbors = sorted(
            neighbors, key=lambda n: similarity[start, n]
        )[:num_neighbors]
        for n in neighbors:
            if n not in visited:
                current.append(n)
                visited.add(n)

        subgraphs.append(current)

    # Build inter-subgraph connection matrix
    num_subs = len(subgraphs)
    connection_matrix = np.zeros((num_subs, num_subs))
    node_to_subgraph = {}
    for i, sub in enumerate(subgraphs):
        for node in sub:
            node_to_subgraph[node] = i

    for edge in G.edges():
        a, b = edge
        if a in node_to_subgraph and b in node_to_subgraph:
            i = node_to_subgraph[a]
            j = node_to_subgraph[b]
            if i != j:
                connection_matrix[i][j] += 1

    return subgraphs, connection_matrix

In [21]:
from torch_geometric.datasets import Planetoid
from torch_geometric.utils import to_dense_adj
import torch
import numpy as np

dataset = Planetoid(root='/tmp/Cora', name='Cora')
data = dataset[0]


In [28]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
adj_matrix = to_dense_adj(data.edge_index)[0].numpy()
features = data.x.numpy()
labels = data.y.numpy()
num_classes = dataset.num_classes


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCN(in_channels=data.num_features, hidden_channels=64, out_channels=dataset.num_classes, num_layers=2).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
criterion = nn.NLLLoss()

subgraphs, conn_matrix = build_subgraphs(data, subgraph_size=5, num_neighbors=4)

for epoch in range(1, 201):
    loss = train(model, data, optimizer, criterion, subgraphs, device)
    acc, f1 = test(model, data, device)
    print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Acc: {acc:.4f} | F1: {f1:.4f}")


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got -2)

In [32]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.utils import to_networkx, to_dense_adj
import networkx as nx
import numpy as np
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity

class GCN(nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels, num_layers):
        super().__init__()
        self.convs = nn.ModuleList()
        self.convs.append(nn.Linear(in_channels, hidden_channels))
        for _ in range(num_layers - 2):
            self.convs.append(nn.Linear(hidden_channels, hidden_channels))
        self.convs.append(nn.Linear(hidden_channels, out_channels))

    def forward(self, x, adjs):
        # adjs is a list of sparse adjacency matrices (one per conv layer except last)
        for conv, adj in zip(self.convs[:-1], adjs):
            x = conv(x)
            x = torch.sparse.mm(adj, x)
            x = F.relu(x)
        x = self.convs[-1](x)
        return F.log_softmax(x, dim=-1)

def build_subgraphs(data, subgraph_size=5, num_neighbors=4):
    G = to_networkx(data, to_undirected=True)
    features = data.x.cpu().numpy()
    visited = set()
    subgraphs = []

    similarity = cosine_similarity(features)

    while len(visited) < data.num_nodes:
        available_nodes = list(set(range(data.num_nodes)) - visited)
        if not available_nodes:
            break

        start = np.random.choice(available_nodes)
        current = [start]
        visited.add(start)

        neighbors = list(G.neighbors(start))
        neighbors = sorted(neighbors, key=lambda n: similarity[start, n])[:num_neighbors]
        for n in neighbors:
            if n not in visited:
                current.append(n)
                visited.add(n)

        subgraphs.append(current)

    # Build inter-subgraph connection matrix (optional)
    num_subs = len(subgraphs)
    connection_matrix = np.zeros((num_subs, num_subs))
    node_to_subgraph = {}
    for i, sub in enumerate(subgraphs):
        for node in sub:
            node_to_subgraph[node] = i

    for a, b in G.edges():
        if a in node_to_subgraph and b in node_to_subgraph:
            i = node_to_subgraph[a]
            j = node_to_subgraph[b]
            if i != j:
                connection_matrix[i, j] += 1

    return subgraphs, connection_matrix

def build_subgraph_adj(data, sub_nodes, device):
    # Build adjacency for subgraph nodes with remapped indices
    edge_index = data.edge_index.cpu().numpy()
    mask = np.isin(edge_index[0], sub_nodes) & np.isin(edge_index[1], sub_nodes)
    filtered_edges = edge_index[:, mask]

    # Remap global node indices to subgraph local indices
    node_map = {n: i for i, n in enumerate(sub_nodes)}
    remapped_edges = np.array([[node_map[n] for n in filtered_edges[0]],
                              [node_map[n] for n in filtered_edges[1]]])

    # Create sparse adjacency matrix (COO)
    indices = torch.tensor(remapped_edges, dtype=torch.long).to(device)
    values = torch.ones(indices.shape[1], device=device)
    adj = torch.sparse_coo_tensor(indices, values, (len(sub_nodes), len(sub_nodes))).coalesce()
    return adj

def train(model, data, optimizer, criterion, subgraphs, device):
    model.train()
    total_loss = 0
    for sub_nodes in subgraphs:
        x = data.x[sub_nodes].to(device)
        y = data.y[sub_nodes].to(device)
        adj = build_subgraph_adj(data, sub_nodes, device)
        # Repeat adjacency matrix for each layer except the last
        adjs = [adj] * (len(model.convs) - 1)

        optimizer.zero_grad()
        out = model(x, adjs)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(subgraphs)

def test(model, data, device):
    model.eval()
    with torch.no_grad():
        x = data.x.to(device)
        edge_index = data.edge_index.to(device)
        values = torch.ones(edge_index.shape[1], device=device)
        adj = torch.sparse_coo_tensor(edge_index, values, (x.size(0), x.size(0))).coalesce()
        adjs = [adj] * (len(model.convs) - 1)

        out = model(x, adjs)
        pred = out.argmax(dim=1)
        true = data.y.to(device)

        acc = accuracy_score(true.cpu(), pred.cpu())
        f1 = f1_score(true.cpu(), pred.cpu(), average='micro')
        return acc, f1

# Usage example (make sure you have dataset & data loaded properly):
subgraphs, conn_matrix = build_subgraphs(data, subgraph_size=5, num_neighbors=4)

res=[]
for i in range(10):
  for epoch in range(1, 101):
      loss = train(model, data, optimizer, criterion, subgraphs, device)
      acc, f1 = test(model, data, device)
      print(f"Epoch {epoch:03d} | Loss: {loss:.4f} | Acc: {acc:.4f} | F1: {f1:.4f}")
  acc, f1 = test(model, data, device)
  res.append(acc)

Epoch 001 | Loss: 1.2516 | Acc: 0.8449 | F1: 0.8449
Epoch 002 | Loss: 1.0687 | Acc: 0.8678 | F1: 0.8678
Epoch 003 | Loss: 1.0615 | Acc: 0.8264 | F1: 0.8264
Epoch 004 | Loss: 1.1070 | Acc: 0.8608 | F1: 0.8608
Epoch 005 | Loss: 1.0410 | Acc: 0.8589 | F1: 0.8589
Epoch 006 | Loss: 1.0471 | Acc: 0.8504 | F1: 0.8504
Epoch 007 | Loss: 1.0599 | Acc: 0.8604 | F1: 0.8604
Epoch 008 | Loss: 1.0213 | Acc: 0.8530 | F1: 0.8530
Epoch 009 | Loss: 1.0549 | Acc: 0.8508 | F1: 0.8508
Epoch 010 | Loss: 1.0442 | Acc: 0.8490 | F1: 0.8490
Epoch 011 | Loss: 1.0289 | Acc: 0.8194 | F1: 0.8194
Epoch 012 | Loss: 1.0429 | Acc: 0.8552 | F1: 0.8552
Epoch 013 | Loss: 1.0287 | Acc: 0.8353 | F1: 0.8353
Epoch 014 | Loss: 1.0337 | Acc: 0.8541 | F1: 0.8541
Epoch 015 | Loss: 1.0337 | Acc: 0.8538 | F1: 0.8538
Epoch 016 | Loss: 1.0300 | Acc: 0.8541 | F1: 0.8541
Epoch 017 | Loss: 1.0378 | Acc: 0.8586 | F1: 0.8586
Epoch 018 | Loss: 1.0267 | Acc: 0.8043 | F1: 0.8043
Epoch 019 | Loss: 1.0760 | Acc: 0.8571 | F1: 0.8571
Epoch 020 | 

In [33]:
print(res)
print(sum(res)/10)

[0.8578286558345642, 0.8574593796159528, 0.869645494830133, 0.8585672082717873, 0.8714918759231906, 0.8685376661742984, 0.8463810930576071, 0.8574593796159528, 0.8508124076809453, 0.8545051698670606]
0.8592688330871493


In [34]:


print(f"Allocated memory : {torch.cuda.memory_allocated() / (1024**2):.2f} MB")
print(f"Reserved memory : {torch.cuda.memory_reserved() / (1024**2):.2f} MB")
print(f"Peak allocated memory: {torch.cuda.max_memory_allocated() / (1024**2):.2f} MB")

Allocated memory : 18.38 MB
Reserved memory : 44.00 MB
Peak allocated memory: 37.85 MB
